### Import Libraries 


 - The os module allows us to interact with operating systems including changing working directory
 - The pandas module allows us to read, clean and preprocess the output CSV files that were web scraped
 - The glob module allows us to search through file paths and search for files that meet a specific pattern in this example we look for .csv patterns
 

In [28]:
import glob as gb
import pandas as pd 
import os

### Set Path
- Set your path to where CSV outputs of your zillow and Craigslist webscraping was exported to us the os chdir method
- Here since we are working with two different output files (Zillow and Craigslist data), make sure they are all in one directory

In [5]:
path = "C:\\Users\\padu\\Desktop\\Zillow\\ZillowCraiglist"
os.chdir(path)

### Read Zillow and Craiglist Data
- read individual files for Zillow and Craigslist

In [7]:
Zillow = pd.read_csv('ZillowUnique.csv')
Craiglist = pd.read_csv('FinalCraiglistDate.csv')

In [9]:
len(Craiglist), len(Zillow)

(39490, 16272)

In [10]:
# Zillow prices are listed with the '/mo' sign for monthly rent price
# Since we want only the numeric values of prices, we replace the '/mo' with blank text in a loop
# we append each of the numerica values to a list called 'finalprice'
# we then create a new column in our dataframe called 'finalprice' and pass the 'finalprice' list to the column

finalprice = []
for row in Zillow['price']:
    first = row.replace('/mo','').replace('+','')
    finalprice.append(first)
print(len(finalprice))


Zillow['finalprice']  = finalprice

16272


### Rename columns
- Since we want to combine the Zillow and Craiglist by concatenating them, the columns names should be the same
- we rename craiglist column name to match zillow columns.
- for bedrooms and bathroom, the data from craigslist is combined in the same column and separated by "/"
- wed split the using the str.split() method in pandas, and place them under different columns

In [11]:
Craiglist = Craiglist.rename(columns = {'OverviewClean': 'overview', 'PostURL':'url',
                                        'Price':'finalprice','address':'streetaddress',
                                       'Neighborood': 'Locality',
                                       'lat': 'latitude','lon':'longitude'})

In [12]:
Craiglist[['bedrooms','baths']] = Craiglist.bedroom_baths.str.split("/", expand =True)

In [13]:
#Verify if columns are similar for both Zillow and Craigslist

Craiglist.columns, Zillow.columns

(Index(['url', 'streetaddress', 'bedroom_baths', 'Title', 'finalprice',
        'latitude', 'longitude', 'overview', 'Locality', 'floorsize',
        'bedrooms', 'baths'],
       dtype='object'),
 Index(['latitude', 'longitude', 'floorsize', 'streetaddress', 'zipcode',
        'Locality', 'url', 'price', 'bedrooms', 'bedroomsLab', 'baths',
        'overview', 'finalprice'],
       dtype='object'))

### Concatenate the two dataframe
- concatenate the zillow and craigslist data 
- Delete columns that are no more needed 



In [19]:
Rentals  = pd.concat([Zillow, Craiglist], axis=0)

In [20]:
columns = ['bedroom_baths','bedroomsLab','price']

In [21]:
Rentals = Rentals.drop(columns,axis =1)

In [22]:
# Replace floorsizes that have "available date" with zero

for row in Rentals['floorsize']:
        a = str(row)
        if a.startswith('a'):
            Rentals['floorsize'] = Rentals['floorsize'].replace(str(row),"0")

In [20]:

len(Rentals)

55762

### Check for Duplicates 
- We checked for duplicates in the combined data 
- We checked for duplicates in only streetaddress
- We checked for duplicates in only the description text
- We checked for duplicaes in both the description text and street address
- For our project, we used the data set that had no duplicates in the streetaddress

In [23]:
# We checked for duplicates in the combined data 
Rentals = Rentals.drop_duplicates()

# We checked for duplicates in only streetaddress
StreetAddress =  Rentals.drop_duplicates(subset = "streetaddress")

# We checked for duplicates in only the description text
Overview = Rentals.drop_duplicates(subset = "overview")

#We checked for duplicaes in both the description text and street address
StreetOverview = Rentals.drop_duplicates(subset = ["overview","streetaddress"])

### Export merged data

In [22]:
Rentals.to_csv("FinalZillowCraig_update.csv", index=False)
StreetAddress.to_csv("FinalZillowStreetAddress_update.csv", index=False)
Overview.to_csv("FinalZillowOverview_update.csv", index=False)
StreetOverview.to_csv("FinalStreetOverview_update.csv", index=False)

In [27]:
# Verify datashape  

Rentals.shape, StreetAddress.shape, Overview.shape, StreetOverview.shape

((51155, 12), (10146, 12), (32638, 12), (35172, 12))